In [1]:
# Importing modules
import pandas as pd
import requests, zipfile
from io import BytesIO

In [2]:
# Downloading the data
print("Beginning Download...")

# Defining the zip file URL
url = "https://meps.ahrq.gov/mepsweb/data_files/pufs/h209/h209xlsx.zip"

# Split URL to get the file name
filename = url.split('/')[-1]

# Downloading the file by sending the request to the URL
req = requests.get(url)
print('Download Complete.')

# extracting the zip file contents
zipfile= zipfile.ZipFile(BytesIO(req.content))
zipfile.extractall('./Resources')

Beginning Download...
Download Complete.


In [3]:
# Converting Data from .xlsx to .csv and then to DataFrame
# Initializing needed column list
col_list = ['DUID', 'FAMSZE18', 'REGION18', 'ENDRFY18', 'AGE18X', 'SEX', 'RACEV2X', 'EDUCYR', 'HIDEG', 'TTLP18X', 'ADBMI42', 'ADOFTB42', 'UNINS18', 'TOTEXP18', 'TOTSLF18', 'TOTMCR18', 'TOTMCD18', 'TOTVA18', 'TOTPTR18', 'TOTOTH18'] 

# Reading necessary columns from excel
print("Reading Excel File...")
excel = pd.read_excel('./Resources/h209.xlsx', usecols=col_list)

# Converting to CSV for faster processing
print("Coverting Excel File to CSV...")
excel.to_csv("./Resources/2018_FYC.csv", index=None, header=True)

# Reading CSV into DF
print("Reading CSV into DataFrame...")
df_raw = pd.DataFrame(pd.read_csv("./Resources/2018_FYC.csv"))

# Reviewing DataFrame
print("Conversion Complete.")
df_raw.head()

Reading Excel File...
Coverting Excel File to CSV...
Reading CSV into DataFrame...
Conversion Complete.


,DUID,FAMSZE18,REGION18,ENDRFY18,AGE18X,SEX,RACEV2X,EDUCYR,HIDEG,ADBMI42,ADOFTB42,TTLP18X,UNINS18,TOTEXP18,TOTSLF18,TOTMCR18,TOTMCD18,TOTVA18,TOTPTR18,TOTOTH18
0,2290001,2,2,2018,27,2,1,16,4,21.4,3,32000,2,2368,225,0,2037,0,107,0
1,2290001,2,2,2018,25,1,1,17,4,30.6,3,0,2,2040,136,0,0,0,1904,0
2,2290002,6,2,2018,34,2,1,10,3,28.2,3,25000,2,173,74,0,0,0,99,0
3,2290002,6,2,2018,39,1,1,10,3,28.7,3,30000,2,0,0,0,0,0,0,0
4,2290002,6,2,2018,11,1,1,4,8,-1.0,-1,0,2,103,69,0,0,0,34,0


In [23]:
# Cleaning Data

# Renaming columns
df_raw.columns = ['ID', 'Family_Size', 'Region', 'Year', 'Age', 'Sex', 'Race', 'Education_Level', 'Highest_Degree', 'BMI', 'Tobacco_Use', 'Total_Personal_Income', 'Uninsured_2018', 'Total_Expenditure', 'Exp_Pocket', 'Exp_Medicare', 'Total_Medicaid', 'Exp_VA', 'Total_Priv_Tri', 'Total_Other']
         
# Filtering data to just the adults
df_trim = df_raw.loc[(df_raw['Age'] > 20)]

# Filtering only for insured customers
df_trim = df_trim.loc[(df_trim['Uninsured_2018'] == 2)]

# Filtering for all valid family sizes
df_trim = df_trim.loc[(df_trim['Family_Size'] >= 0)]

# Filtering for all valid BMI 
df_trim = df_trim.loc[(df_trim['BMI'] > 5)]

# Filtering for Total_Expenditure outliers
df_trim = df_trim.loc[(df_trim['Total_Expenditure'] < 200000)]

# Filtering for applicable tobacco answers (1=Yes/2=No)
df_trim = df_trim.loc[(df_trim['Tobacco_Use'] > 0)]

# Removing unnecessary columns
df_trim.drop('Year', axis=1, inplace=True)
df_trim.drop('Uninsured_2018', axis=1, inplace=True)

# Reviewing Dataframe
df_trim

,ID,Family_Size,Region,Age,Sex,Race,Education_Level,Highest_Degree,BMI,Tobacco_Use,Total_Personal_Income,Total_Expenditure,Exp_Pocket,Exp_Medicare,Total_Medicaid,Exp_VA,Total_Priv_Tri,Total_Other
0,2290001,2,2,27,2,1,16,4,21.4,3,32000,2368,225,0,2037,0,107,0
1,2290001,2,2,25,1,1,17,4,30.6,3,0,2040,136,0,0,0,1904,0
2,2290002,6,2,34,2,1,10,3,28.2,3,25000,173,74,0,0,0,99,0
3,2290002,6,2,39,1,1,10,3,28.7,3,30000,0,0,0,0,0,0,0
8,2290003,4,2,36,2,1,17,5,21.5,3,30217,535,232,0,0,0,303,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30455,2329684,2,3,59,2,1,16,4,34.3,3,43670,2953,966,0,0,0,1987,0
30456,2329684,2,3,66,1,1,16,4,31.0,2,8999,14660,9288,18,0,0,5354,0
30457,2329685,1,3,69,2,2,17,5,45.7,3,55132,2439,859,1265,0,0,313,0
30458,2329687,3,4,33,2,1,16,4,29.2,3,18000,17820,2114,0,0,0,15706,0


In [24]:
# Trasnferring DataFrame to CSV for mutual use in project
df_trim.to_csv("./Resources/2018_FYC.csv", index=None, header=True)